In [1]:
import sqlalchemy as sa
import pandas as pd
import requests, json
import datetime as dt

In [2]:
BASE_URL='https://api.openweathermap.org/data/2.5/weather?'
API_KEY='2e97c849f19a87086e65d9add05608c6'
#Asking user to input the name of they want current weather info for
CITY=input("Enter the city name: ") 

url=BASE_URL + 'appid='+ API_KEY + "&q=" + CITY

data=requests.get(url)


Enter the city name: Ottawa


In [3]:
data

<Response [200]>

In [4]:
data=data.json()
data

{'coord': {'lon': -75.6981, 'lat': 45.4112},
 'weather': [{'id': 802,
   'main': 'Clouds',
   'description': 'scattered clouds',
   'icon': '03d'}],
 'base': 'stations',
 'main': {'temp': 296.33,
  'feels_like': 296.22,
  'temp_min': 295.31,
  'temp_max': 297.13,
  'pressure': 1008,
  'humidity': 58},
 'visibility': 10000,
 'wind': {'speed': 5.66, 'deg': 270},
 'clouds': {'all': 40},
 'dt': 1691790427,
 'sys': {'type': 2,
  'id': 2005537,
  'country': 'CA',
  'sunrise': 1691747912,
  'sunset': 1691799460},
 'timezone': -14400,
 'id': 6094817,
 'name': 'Ottawa',
 'cod': 200}

Checking to see what information could be relevant for current weather data

In [5]:
data['main']

{'temp': 296.33,
 'feels_like': 296.22,
 'temp_min': 295.31,
 'temp_max': 297.13,
 'pressure': 1008,
 'humidity': 58}

In [6]:
#Converting kelvin to celsius:

def kelvin_to_celsius(kelvin):
    celsius=kelvin-273.15
    return celsius


We first need to obtain current date time for quarry date and time columns. Then we chose name of city, country of city, temp, feels like, wind speed, humidity, sunrise time and sunset time as relevant columns for the current weather table.

In [7]:
current_date_time = dt.datetime.now()
current_date_formatted = current_date_time.strftime('%Y-%m-%d')
current_time_formatted = current_date_time.strftime('%H:%M:%S')

name=data['name']
country=data['sys']['country']
temp_kelvin=data['main']['temp']
temp_celsius=kelvin_to_celsius(temp_kelvin)
feels_like_kelvin=data['main']['feels_like']
feels_like_celsius=kelvin_to_celsius(feels_like_kelvin)
wind_speed=data['wind']['speed']
humidity=data['main']['humidity']
description=data['weather'][0]['description']

sunrise_time=dt.datetime.utcfromtimestamp(data['sys']['sunrise']+data['timezone'])
sunrise_formatted_time = sunrise_time.strftime('%H:%M:%S')
sunset_time=dt.datetime.utcfromtimestamp(data['sys']['sunset']+data['timezone'])
sunset_formatted_time = sunset_time.strftime('%H:%M:%S')

Building a data dictionary w/ relevant columns and converting it to a pandas dataframe

In [8]:
data_dict = {
    'Query Date': [current_date_formatted],
    'Query Time': [current_time_formatted],
    'City Name':[name],
    'Country':[country],
    'Temperature (C)': [temp_celsius],
    'Feels Like (C)': [feels_like_celsius],
    'Wind Speed(m/s)': [wind_speed],
    'Humidity (%)': [humidity],
    'Description': [description],
    'Sunrise Time': [sunrise_formatted_time],
    'Sunset Time': [sunset_formatted_time]
}

df = pd.DataFrame(data_dict)
print(df)

   Query Date Query Time City Name Country  Temperature (C)  Feels Like (C)  \
0  2023-08-11   17:49:56    Ottawa      CA            23.18           23.07   

   Wind Speed(m/s)  Humidity (%)       Description Sunrise Time Sunset Time  
0             5.66            58  scattered clouds     05:58:32    20:17:40  


In [9]:
my_db_secret = {
    'drivername' : 'postgresql+psycopg2',
    'host'       : 'mmai5100postgres.canadacentral.cloudapp.azure.com',
    'port'       : '5432',
    'username'   : 'nashaatk',
    'password'   : '2023!Schulich',
    'database'   : 'nashaatk_db'
}

In [10]:
my_db_url = sa.engine.URL.create(
    drivername = my_db_secret['drivername'],
    username   = my_db_secret['username'],
    password   = my_db_secret['password'],
    host       = my_db_secret['host'],
    port       = my_db_secret['port'],
    database   = my_db_secret['database']
)

In [11]:
my_db_engine = sa.create_engine(my_db_url)

Pushing the info into sql database

In [12]:
df.to_sql(
    name = 'current_weather',
    schema = 'mmai_is_cool',
    con = my_db_engine,
    if_exists = 'append',
    index=False,
    method='multi',
    dtype= {
        'Query Date': sa.types.DATE,
        'Query Time':sa.types.TIME,
        'City Name':sa.types.VARCHAR(225),
        'Country':sa.types.VARCHAR(225),
        'Temperature (C)': sa.types.DECIMAL(5,2),
        'Feels Like (C)': sa.types.DECIMAL(5,2),
        'Wind Speed(m/s)': sa.types.DECIMAL(6,2),
        'Humidity (%)': sa.types.DECIMAL(4,1),
        'Description': sa.types.VARCHAR(225),
        'Sunrise Time': sa.types.TIME,
        'Sunset Time': sa.types.TIME,
        
    }   
)